# Práctica 3: Representación del conocimiento

__Fecha de entrega: 23 de febrero de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: XX__

__Nombres de los estudiantes: Gonzalo García Fernández y Daniel Carreño López__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [1]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.5 MB/s eta 0:00:00


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [ ]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q13050139,Q13050139
3,http://www.wikidata.org/entity/Q23003932,His Whiskers
4,http://www.wikidata.org/entity/Q24287064,Taffy
5,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
6,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
7,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[U2 (Q396)](https://www.wikidata.org/wiki/Q396) es una banda de rock con una larga trayectoria y numerosos éxitos. 

Vamos a comenzar por averiguar su fecha y lugar de formación (localidad y país).

In [59]:
%%wdsparql
SELECT ?fecha ?localidad ?localidadLabel ?pais ?paisLabel 
WHERE 
{
    wd:Q396 wdt:P571 ?fecha. #fecha de formacion de U2
    wd:Q396 wdt:P740 ?localidad.#lugar de formacion(ciudad)
    ?localidad wdt:P17 ?pais.#pais de la ciudad
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,fecha,localidad,localidadLabel,pais,paisLabel
0,1976-01-01T00:00:00Z,http://www.wikidata.org/entity/Q1761,Dublin,http://www.wikidata.org/entity/Q27,Republic of Ireland


### Consulta 2

A continuación vamos a averiguar todos los miembros que se le conocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por su apellido e incluir su nombre completo y el artístico.

Puede que la siguiente información sobre el uso de [OPTIONAL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [73]:
%%wdsparql
SELECT ?miembro ?miembroLabel ?nombre ?alias 
WHERE
{
  wd:Q396 wdt:P527 ?miembro.
  OPTIONAL{?miembro wdt:P1477 ?nombre.}#Nombre completo lo ponemos opcional porque no hay nombre en uno de ellos por wikidata
 OPTIONAL { ?miembro wdt:P742 ?alias. } #alias que solo the EDGE y Bono tienen
 SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?apellido) 

,miembro,miembroLabel,nombre,alias
0,http://www.wikidata.org/entity/Q175907,Adam Clayton,Adam Charles Clayton,None
1,http://www.wikidata.org/entity/Q714845,The Edge,David Howell Evans,The Edge
2,http://www.wikidata.org/entity/Q2136220,Dik Evans,None,None
3,http://www.wikidata.org/entity/Q814065,Larry Mullen Jr.,Larry Mullen Jr,None
4,http://www.wikidata.org/entity/Q834621,Bono,Paul David Hewson,Bono


Es importante usar dos OPTIONAL distintos para que se muestren todos los miembros, tengan o no pseudónimo y nombre completo asociados.

### Consulta 3

[Bono (Q834621)](http://www.wikidata.org/entity/Q834621) es el cantante de la banda. Además de cantante, ha tenido muchas otras ocupaciones. ¿Cuáles de ellas están relacionadas directamente con el mundo del [arte (Q483501](https://www.wikidata.org/wiki/Q483501)? Ten en cuenta que la jerarquía de ocupaciones puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [62]:
%%wdsparql
SELECT ?ocupacion ?ocupacionLabel
WHERE 
{
    wd:Q834621 wdt:P106 ?ocupacion.#sus ocupaciones 
    ?ocupacion wdt:P279* wd:Q483501.#esten realcionados con arte de alguna manera
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,ocupacion,ocupacionLabel
0,http://www.wikidata.org/entity/Q28389,screenwriter
1,http://www.wikidata.org/entity/Q33999,actor
2,http://www.wikidata.org/entity/Q488205,singer-songwriter
3,http://www.wikidata.org/entity/Q855091,guitarist


Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

### Consulta 4

Bono se ha casado una vez, con Ali Hewson, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento. Los resultados deben aparecer ordenados cronológicamente.

In [64]:
%%wdsparql
SELECT ?nombrehijo ?nombrehijoLabel ?fecha
WHERE
{
  wd:Q834621 wdt:P40 ?nombrehijo. #sea hijo de bono
  ?nombrehijo wdt:P569 ?fecha. #fecha de nacimiento
 SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha)

,nombrehijo,nombrehijoLabel,fecha
0,http://www.wikidata.org/entity/Q3735288,Eve Hewson,1991-07-07T00:00:00Z
1,http://www.wikidata.org/entity/Q82782709,Elijah Hewson,1999-08-18T00:00:00Z


### Consulta 5

Bono ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible). Los resultados se deben mostrar ordenado por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [65]:
%%wdsparql
SELECT ?premio ?premioLabel ?fecha
WHERE
{
  wd:Q834621 p:P166 ?st.
  ?st ps:P166 ?premio.#que premio es
  OPTIONAL { ?st pq:P585 ?fecha. }#en que año lo ganó
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha)

,premio,premioLabel,fecha
0,http://www.wikidata.org/entity/Q41254,Grammy Awards,None
1,http://www.wikidata.org/entity/Q52382875,Fellow of the American Academy of Arts and Sci...,None
2,http://www.wikidata.org/entity/Q326482,MusiCares Person of the Year,2003-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q10855271,Knight of the Legion of Honour,2003-01-01T00:00:00Z
4,http://www.wikidata.org/entity/Q23014208,TED Prize,2005-01-01T00:00:00Z
5,http://www.wikidata.org/entity/Q179191,Rock and Roll Hall of Fame,2005-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q67155276,Officer of the Order of Liberty,2005-04-21T00:00:00Z
7,http://www.wikidata.org/entity/Q207826,Time Person of the Year,2005-12-19T00:00:00Z
8,http://www.wikidata.org/entity/Q7182737,Philadelphia Liberty Medal,2007-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q17011862,NAACP Image Award – Chairman's Award,2007-01-01T00:00:00Z


### Consulta 6

Ahora queremos conocer todos los discos de U2 del tipo [albunes en directo (Q209939)](https://www.wikidata.org/wiki/Q209939) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre e incluir el año de publicación.

In [67]:
%%wdsparql
SELECT ?album ?albumLabel  ?anyo (MIN(?generoLabel) AS ?gen) #solo mostramos un genero porque dice cualquiera de ellos
WHERE
{
?album wdt:P175 wd:Q396.#algo que tiene como performer U2
?album wdt:P7937* wd:Q209939.# albumes en directo
OPTIONAL{?album wdt:P577 ?fecha.#la fecha de cada album
  BIND(YEAR(?fecha) as ?anyo).}#sacamos el año
OPTIONAL{?album wdt:P136 ?genero.}#uno de los subgeneros del album
SERVICE wikibase:label { bd:serviceParam wikibase:language "en".
                        ?album rdfs:label ?albumLabel.
                        ?genero rdfs:label ?generoLabel.}
}
GROUP BY ?album ?albumLabel ?anyo
ORDER BY ASC(?albumLabel)

,album,albumLabel,anyo,gen
0,http://www.wikidata.org/entity/Q2393675,Elevation 2001: Live from Boston,NaN,rock music
1,http://www.wikidata.org/entity/Q3719109,From the Ground Up: Edge's Picks from U2360°,2012.0,None
2,http://www.wikidata.org/entity/Q2400896,Hasta la Vista Baby!,NaN,rock music
3,http://www.wikidata.org/entity/Q2733732,Live from Paris,NaN,rock music
4,http://www.wikidata.org/entity/Q3280083,Live from the Point Depot,2004.0,rock music
5,http://www.wikidata.org/entity/Q3908356,PopMart: Live from Mexico City,NaN,rock music
6,http://www.wikidata.org/entity/Q478327,Rattle and Hum,1988.0,blues rock
7,http://www.wikidata.org/entity/Q387595,The Complete U2,2004.0,rock music
8,http://www.wikidata.org/entity/Q1764703,"U2 Go Home: Live from Slane Castle, Ireland",2003.0,rock music
9,http://www.wikidata.org/entity/Q2568650,U2.COMmunication,2005.0,rock music


In [87]:
#Esta solucion es mas eficiente pero solo sale un resultado entrando en la discografía, en las listas y luego ya en el porpio album en directo con form of creative work.
%%wdsparql
SELECT ?album ?albumLabel  ?anyo (MIN(?generoLabel) AS ?gen)
WHERE
{
wd:Q396 wdt:P358 ?disc.
?disc wdt:P2354 ?lista.
?lista wdt:P527 ?album.
 
?album wdt:P7937* wd:Q209939.
OPTIONAL{?album wdt:P577 ?fecha.
  BIND(YEAR(?fecha) as ?anyo).}
OPTIONAL{?album wdt:P136 ?genero.}
SERVICE wikibase:label { bd:serviceParam wikibase:language "en".
                        ?album rdfs:label ?albumLabel.
                        ?genero rdfs:label ?generoLabel.}
}
GROUP BY ?album ?albumLabel ?anyo
ORDER BY ASC(?albumLabel)

,album,albumLabel,anyo,gen
0,http://www.wikidata.org/entity/Q478327,Rattle and Hum,1988,blues rock


### Consulta 7

Queremos saber si hay obras de U2 publicadas varias veces (en diferentes años). Para ello vamos a recuperar las obras de tipo [lanzamiento de un grupo (group release)](https://www.wikidata.org/wiki/Q108346082) pero mostrando también la fecha de publicación más antigua de cada una y el número de apariciones. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por album y aplicar una función de agregación sobre las fechas de lanzamiento. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [69]:
%%wdsparql
SELECT ?album ?albumLabel (COUNT(*) AS ?apariciones) (MIN (?fecha) AS ?primerlanzamiento) 
WHERE
{
  ?album wdt:P175 wd:Q396;#algo que tiene como performer U2
         wdt:P7937/wdt:P279* wd:Q108346082.#sea un group release o una subclase de ello
  OPTIONAL { ?album wdt:P577 ?fecha. }#fecha
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?album ?albumLabel
ORDER BY ?primerlanzamiento

,album,albumLabel,apariciones,primerlanzamiento
0,http://www.wikidata.org/entity/Q2393675,Elevation 2001: Live from Boston,1,None
1,http://www.wikidata.org/entity/Q2400896,Hasta la Vista Baby!,1,None
2,http://www.wikidata.org/entity/Q2733732,Live from Paris,1,None
3,http://www.wikidata.org/entity/Q3908356,PopMart: Live from Mexico City,1,None
4,http://www.wikidata.org/entity/Q207662,Boy,1,1980-10-20T00:00:00Z
5,http://www.wikidata.org/entity/Q269674,October,1,1981-10-12T00:00:00Z
6,http://www.wikidata.org/entity/Q331811,War,1,1983-02-28T00:00:00Z
7,http://www.wikidata.org/entity/Q221116,Under a Blood Red Sky,1,1983-11-21T00:00:00Z
8,http://www.wikidata.org/entity/Q536336,The Unforgettable Fire,1,1984-10-01T00:00:00Z
9,http://www.wikidata.org/entity/Q9358291,The Unforgettable Fire Collection,1,1985-01-01T00:00:00Z


### Consulta 8

U2 es sin duda un grupo con una larga trayectoria. ¿Cuántas discos ha lanzado? Vamos a considerar sólo instancias directas de [album de estudio (Q208569)](https://www.wikidata.org/wiki/Q208569).

In [81]:
%%wdsparql
SELECT (COUNT(DISTINCT ?album) as ?c)
WHERE
{
?album wdt:P175 wd:Q396.#algo que tiene como performer U2
?album wdt:P7937 wd:Q208569.#que sea un album de estudio
}

,c
0,14


### Consulta 9

Entre todos sus discos de estudio, queremos recuperar los títulos, fechas de lanzamiento y discográfica de los discos de la década de los 1990. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [76]:
%%wdsparql
SELECT ?album ?albumLabel ?fecha
WHERE
{
  ?album wdt:P175 wd:Q396.#algo que tiene como performer U2
  ?album wdt:P7937 wd:Q208569.#que sea un album de estudio
  ?album wdt:P577 ?fecha.#fecha no opcional porque nos hace falta
  FILTER("1990-01-01"^^xsd:dateTime <= ?fecha && ?fecha < "2000-01-01"^^xsd:dateTime).#que sean entre 1990 y 1999 ambos inclusive
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha)

,album,albumLabel,fecha
0,http://www.wikidata.org/entity/Q308801,Achtung Baby,1991-11-18T00:00:00Z
1,http://www.wikidata.org/entity/Q220336,Zooropa,1993-07-05T00:00:00Z
2,http://www.wikidata.org/entity/Q731908,Pop,1997-03-03T00:00:00Z


### Consulta 10

¿Qué otros personajes famosos nacieron el mismo año que Bono y tiene su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre y fecha de nacimiento.

In [83]:
%%wdsparql
SELECT DISTINCT ?famoso ?famosoLabel ?nacimiento (GROUP_CONCAT(?ocupacionLabel) AS ?ocps)#mostramos todas las posibles 
WHERE
{
  wd:Q834621 wdt:P569 ?nacimientobono.#fecha de nacimiento de bono
  wd:Q834621 wdt:P27 ?paisbono.#nacionalidad de bono
  
  ?famoso wdt:P569 ?nacimiento.#nacimiento de los famosos
  FILTER(YEAR(?nacimientobono) = YEAR(?nacimiento)).#filtramos que nazcan el mismo año que Bono
  FILTER(?famoso != wd:Q834621).#que no sean Bono
 
  ?famoso wdt:P27 ?paisbono;#que tengan la misma nacionalidad que bono
          wdt:P106 ?ocupacion.#miramos su ocupacion
 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en".
                          ?famoso rdfs:label ?famosoLabel.
                          ?ocupacion rdfs:label ?ocupacionLabel.}
}
GROUP BY ?famoso ?famosoLabel ?nacimiento #agrupamos por nombre de famoso y fecha de nacimiento
ORDER BY ASC (?famosoLabel)(?nacimiento)

,famoso,famosoLabel,nacimiento,ocps
0,http://www.wikidata.org/entity/Q4706317,Alan Campbell,1960-08-10T00:00:00Z,association football player
1,http://www.wikidata.org/entity/Q2830507,Alan Glynn,1960-01-01T00:00:00Z,screenwriter writer journalist science fiction...
2,http://www.wikidata.org/entity/Q4707358,Alan Moore,1960-02-02T00:00:00Z,poet
3,http://www.wikidata.org/entity/Q68056008,Amira Ghazalla,1960-01-01T00:00:00Z,actor stage actor film actor
4,http://www.wikidata.org/entity/Q4756838,Andrew Doyle,1960-07-02T00:00:00Z,politician farmer
5,http://www.wikidata.org/entity/Q65554176,Ann Hawkshaw,1960-01-01T00:00:00Z,badminton player
6,http://www.wikidata.org/entity/Q55739805,Ann O'Sullivan,1960-01-01T00:00:00Z,badminton player
7,http://www.wikidata.org/entity/Q4823005,Austin Buckley,1960-01-01T00:00:00Z,hurler
8,http://www.wikidata.org/entity/Q4892803,Bernie Nolan,1960-10-17T00:00:00Z,actor singer autobiographer
9,http://www.wikidata.org/entity/Q2607154,Bernie Slaven,1960-11-13T00:00:00Z,association football player autobiographer


### Consulta 11

¿Cuántos asertos hay sobre U2 en Wikidata? Ten en cuenta que puede aparece tanto como sujeto como objeto de cada tripleta.

In [88]:
%%wdsparql
SELECT (COUNT(*) as ?c)
WHERE{
{
wd:Q396 ?propiedad ?objeto. #aquellos en los que U2 sale como sujeto
}
UNION
{
?sujeto ?propiedad wd:Q396. #aquellos en los que U2 sale como objeto
}
filter(not exists{wd:Q396 ?propiedad wd:Q396}). #aquellos que en los que U2 sale como sujeto y objeto para no contralos 2 veces

}

,c
0,1432


__Fecha de las consultas: 21 de abril de 2022__